In [16]:
pip install keras-tuner

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### Read the data

In [18]:
train_data = pd.read_csv('processed-data/train_data.csv')
test_data = pd.read_csv('processed-data/test_data.csv')

### Select input(X) and target(y) variables

In [19]:
X_train = train_data[['input_voltage']].values
y_train = train_data['el_power'].values
X_test = test_data[['input_voltage']].values
y_test = test_data['el_power'].values

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

### Reshape the data for time-series

In [21]:
def create_sequences(data, target, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(target[i + window_size])
    return np.array(X), np.array(y)

# Usjng the last 10 steps to predict the next
window_size = 10

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, window_size)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, window_size)

# Check new shape: (samples, timesteps, feature)
print(X_train_seq.shape) # e.g., (n_samples, 10, m_features)

(52930, 10, 1)


In [24]:
from tensorflow.keras.layers import LSTM, GRU

# Define the RNN model-building function for KerasTuner
def build_rnn_model(hp):
    model = Sequential()

    # Choose between LSTM and GRU using a string choice
    rnn_type = hp.Choice('rnn_type', ['LSTM', 'GRU'])

    if rnn_type == 'LSTM':
        model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32),
                       activation='tanh', 
                       return_sequences=False,  # Set to True if adding more RNN layers
                       input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    else:
        model.add(GRU(units=hp.Int('units', min_value=32, max_value=256, step=32),
                      activation='tanh', 
                      return_sequences=False, 
                      input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])))
    
    # Add dropout after RNN layer
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense layer (optional)
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))

    # Output layer for regression
    model.add(Dense(1))  # No activation since it's regression

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [0.001, 0.0001])),
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    return model


### Set-up the Tuner

In [25]:
tuner = kt.RandomSearch(
    build_rnn_model,
    objective = kt.Objective('val_root_mean_squared_error', direction='min'), # out goal is to mnimize RMSE
    max_trials = 10, # Number of different hyperparameter combinations to try
    executions_per_trial = 2, # Number of models to train with each set of hyperparameters
    directory = 'models/', # save tuning results
    project_name = 'gas_turbine_prediction' # Project name for organizing results
)

In [ ]:
tuner.search(X_train_seq, y_train_seq,
            epochs = 50,
            batch_size = 32,
            validation_split = 0.2)

Trial 2 Complete [00h 39m 45s]
val_root_mean_squared_error: 68.71615219116211

Best val_root_mean_squared_error So Far: 68.71615219116211
Total elapsed time: 01h 22m 12s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
GRU               |LSTM              |rnn_type
96                |160               |units
0.3               |0.2               |dropout
64                |96                |dense_units
0.0001            |0.001             |learning_rate

Epoch 1/50
1324/1324 [==============================] - 32s 20ms/step - loss: 3776037.2500 - root_mean_squared_error: 1943.2029 - val_loss: 3182436.0000 - val_root_mean_squared_error: 1783.9384
Epoch 2/50
1324/1324 [==============================] - 25s 19ms/step - loss: 3118089.7500 - root_mean_squared_error: 1765.8114 - val_loss: 2223543.2500 - val_root_mean_squared_error: 1491.1550
Epoch 3/50
1324/1324 [==============================] - 25s 19ms/step - loss: 2197911.2500 - root_mean_squared_error: 1482

1324/1324 [==============================] - 26s 20ms/step - loss: 140721.5312 - root_mean_squared_error: 375.1287 - val_loss: 9853.6309 - val_root_mean_squared_error: 99.2655
Epoch 42/50
1324/1324 [==============================] - 26s 20ms/step - loss: 139382.3906 - root_mean_squared_error: 373.3395 - val_loss: 12038.5371 - val_root_mean_squared_error: 109.7203
Epoch 43/50
1324/1324 [==============================] - 22s 17ms/step - loss: 138861.4844 - root_mean_squared_error: 372.6412 - val_loss: 13070.5479 - val_root_mean_squared_error: 114.3265
Epoch 44/50
1324/1324 [==============================] - 24s 18ms/step - loss: 139729.1875 - root_mean_squared_error: 373.8037 - val_loss: 9996.7559 - val_root_mean_squared_error: 99.9838
Epoch 45/50
1324/1324 [==============================] - 25s 19ms/step - loss: 140006.0469 - root_mean_squared_error: 374.1738 - val_loss: 11556.7627 - val_root_mean_squared_error: 107.5024
Epoch 46/50
1324/1324 [==============================] - 25s 19ms/

### Retrieve and Evaluate the Best Model

In [ ]:
# Retrieve the best model and its hyperparameters
best_rnn_model = tuner.get_best_models(num_models=1)[0]

# Display the best hyperparameters
best_hps_rnn = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps_rnn.values}")

# Evaluate the best model on the test set
test_loss, test_rmse = best_rnn_model.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f'Test RMSE: {test_rmse:.2f}')


### Fine-Tune the Best Model

In [ ]:
# Optionally, retrain the best model with the full training set
best_rnn_model.fit(X_train_seq, y_train_seq, epochs=100, batch_size=32, validation_split=0.2)